# 李代数基本运算

- python                    Python 3.9.12
- scipy                     1.10.0
- numpy                     1.23.5

In [ ]:
import numpy as np
from scipy.linalg import expm, logm
from numpy import pi, sin, cos, tan, arccos, trace
from numpy.linalg import norm, det, inv

np.set_printoptions(precision=3, suppress=True)
deg = pi / 180

- 函数1. 在$\mathfrak{so}(3)$上的李代数向量转换3*3的为反对称矩阵:

In [ ]:
def so3_v2m(vec):
    return np.mat([
        [0, -vec[0, 2], vec[0, 1]],
        [vec[0, 2], 0, -vec[0, 0]],
        [-vec[0, 1], vec[0, 0], 0]
    ])


In [ ]:
# test
vec = np.mat([3,4,5])
print(f'反对称矩阵：\n{so3_v2m(vec)}')

- 函数2. 把$SO(3)$上的特殊正交群（旋转矩阵）转换为三位向量:

In [ ]:
def SO3_m2v(M):
    return np.mat([M[2,1], M[0,2], M[1,0]])

In [ ]:
# test
m = np.mat([[ 0, -5,  4],
 [ 5,  0, -3],
 [-4,  3,  0]])
print(f'向量为：\n {SO3_m2v(m)}')

- 函数3. 在$\mathfrak{se}(3)$上的6维李代数向量转换4*4的矩阵表示形式:

In [ ]:
def se3_v2m(vec):
    return np.mat([
        [0, -vec[0, 2], vec[0, 1], vec[0, 3]],
        [vec[0, 2], 0, -vec[0, 0], vec[0, 4]],
        [-vec[0, 1], vec[0, 0], 0, vec[0, 5]],
        [0, 0, 0, 0]
    ])

In [ ]:
# test
s = np.mat([0.6, 0.3, 0.7416, 4, 5, 6])
theta = pi / 6
sv = theta * s
print(f'向量sv是：\n{sv}')
print(f'4*4矩阵：\n{se3_v2m(sv)}')

- 函数4. 把$SE(3)$上的特殊正交群（旋转矩阵）转换为向量:

In [ ]:
def SE3_m2v(M):
    return np.mat([M[2, 1], M[0, 2], M[1, 0], M[0, 3], M[1, 3], M[2, 3]])

In [ ]:
# test
m = np.mat([[ 0.,    -0.388,  0.157,  2.094],
 [ 0.388,  0.,    -0.314,  2.618],
 [-0.157,  0.314,  0.,     3.142],
 [ 0.,     0.,     0.,     0.   ]])
print(f'向量为：\n {SE3_m2v(m)}')

- 函数5. 通过Tylor展开式化简，将李代数形式的旋转转化为矩阵形式的，即
$$\mathfrak{so}(3) \Rightarrow SO(3)$$

$$\exp(\phi^\wedge)=\cos\theta I+(1-\cos\theta)aa^T+\sin\theta a^\wedge$$

In [ ]:
def so3exp(v):
    I = np.mat(np.eye(3))
    theta = norm(v)
    n = np.mat(v / theta)
    return cos(theta) * I + (1 - cos(theta)) * n.transpose() * n + sin(theta) * so3_v2m(n)

In [ ]:
# test
s = np.mat([0.6, 0.3, 0.7416])
theta = pi/6
sv = theta * s
exp = so3exp(sv)
expp = expm(so3_v2m(sv))
print(f"so3的李代数=\n{sv}")
print(f"指数映射得出=\n{exp}")
print(f"expm计算得出=\n{expp}")

- 函数6. 在$\mathfrak{so}(3)$上一种李括号的定义

In [ ]:
def LieBracket_so3(V1, V2):
    hatV1 = so3_v2m(V1)
    hatV2 = so3_v2m(V2)
    return SO3_m2v(hatV1 * hatV2 - hatV2 * hatV1)

In [ ]:
# test
v1 = np.mat([1,2,3])
v2 = np.mat([2,3,4])
print(f'李括号运算结果为：\n {LieBracket_so3(v1,v2)}')

- 函数7. 在$\mathfrak{se}(3)$上李括号的定义

In [ ]:
def LieBracket_se3(V1, V2):
    hatV1 = se3_v2m(V1)
    hatV2 = se3_v2m(V2)
    return SE3_m2v(hatV1 * hatV2 - hatV2 * hatV1)

In [ ]:
# test
v1 = np.mat([1,2,3,7,8,9])
v2 = np.mat([2,3,4,6.8,7.9,8.1])
print(f'李括号运算结果为：\n {LieBracket_se3(v1,v2)}')

- 函数8. 将李代数形式的位姿转化为矩阵形式的，即
$$\mathfrak{se}(3) \Rightarrow SE(3)$$

In [ ]:
def se3exp(v):
    I = np.mat(np.eye(3))
    theta = norm(v[0, :3])
    n = np.mat(v[0, :3] / theta)
    J = sin(theta) / theta * I + (1 - sin(theta) / theta) * n.transpose() * n + (1 - cos(theta)) / theta * so3_v2m(n)
    R = so3exp(v[0, :3])
    ret = np.hstack((R, J * v[0, 3:6].transpose()))
    ret = np.vstack((ret, np.mat([[0, 0, 0, 1]])))
    return ret

In [ ]:
# test
s = np.mat([0.6, 0.3, 0.7416, 4, 5, 6])
theta = pi / 6
sv = theta * s
exp = se3exp(sv)
expp = expm(se3_v2m(sv))
print(f"so3的李代数=\n{sv}")
print(f"指数映射得出=\n{exp}")
print(f"expm计算得出=\n{expp}")

- 函数9. M为3*3矩阵，解秩为2的齐次线性方程：
$$Mn=0$$
的一个单位解，指向平面$x+y+z=0$向$\sqrt{3}(1,1,1)$方向

p.s.因为对于旋转轴来说，三维空间中只有一个自由度，因此秩为2

In [ ]:
def solveHomoEqu(M):
    A = M[:2, :2]
    B = M[2, :2]
    D = det(A)
    D1 = det(np.vstack((B, A[1, :]))) / D
    D2 = det(np.vstack((A[0, :], B))) / D
    D3 = (-M[0, 0] * D1 - M[0, 1] * D2) / M[0, 2]
    ret = np.mat([[D1, D2, D3]])
    if D1 + D2 + D3 < 0:
        ret = -ret;
    ret = ret / norm(ret)
    return ret

In [ ]:
# test
R = np.mat([[ 0.914, -0.347,  0.21 ],
            [ 0.395,  0.878, -0.27 ],
            [-0.09,   0.330,  0.94 ]])
ans = solveHomoEqu(R-np.eye(3))
print(f'answer is: {ans}')

- 函数10. 将旋转矩阵形式转化为李代数形式的，即

$$SO(3) \Rightarrow \mathfrak{so}(3)$$

$$\theta=\frac{\arccos(trace(R)-1)}{2}$$

且转轴向量满足旋转不变性

$$Rn=n$$

In [ ]:
def SO3log(R):
    theta = arccos((trace(R) - 1) / 2)
    ClamMat = R - np.eye(3)
    n = solveHomoEqu(ClamMat)
    return theta * n

In [ ]:
# test
R = np.mat([[ 0.914, -0.347,  0.21 ],
            [ 0.395,  0.878, -0.27 ],
            [-0.09,   0.330,  0.94 ]])
sv = SO3log(R)
print(f'将特殊正交群转换为李代数形式：v={sv}')

- 函数11. 将位姿矩阵形式转化为李代数形式的，即
$$SE(3) \Rightarrow \mathfrak{se}(3)$$

In [ ]:
def SE3log(T):
    R = T[:3, :3]
    theta = arccos((trace(R) - 1) / 2)
    ClamMat = R - np.eye(3)
    n = solveHomoEqu(ClamMat)
    I = np.mat(np.eye(3))
    J = sin(theta) / theta * I + (1 - sin(theta) / theta) * n.transpose() * n + (1 - cos(theta)) / theta * so3_v2m(n)
    rho = inv(J) * T[0:3, 3]
    return np.hstack((theta*n, rho.transpose()))

In [ ]:
# test
T = np.mat([ [ 0.914, -0.347,  0.21,   1.863],
             [ 0.395,  0.878, -0.27,   2.474],
             [-0.09,   0.33,   0.94,   3.387],
             [ 0.,     0.,     0.,     1.   ]])
sv = SE3log(T)
print(f'将特殊正交群转换为李代数形式：v={sv}')

# 李代数求导与扰动模型

对于$SO(3)$中的两个矩阵乘法，得出的结果对于李代数的改变是什么？

e.g.

对于$A,B\in \mathfrak{so}(3)$，有

$$\ln(\exp(A)\exp(B))^\vee\in\mathfrak{so}(3)$$

然而

$$\ln(\exp(A)\exp(B)) = A+B+\frac{1}{2}[A,B]+\frac{1}{12}[A,[A,B]]-\frac{1}{12}[B,[A,B]]+... $$

当A或B为小量时，二次以上项都会被忽略掉，此时，BCH近似线性表达为：

$$
    \ln(\exp(\phi_1^\wedge)\exp(\phi_2^\wedge))^\vee=\left\{
    \begin{aligned}
    &J_l(\phi_2)^{-1}\phi_1+\phi_2\quad当\phi_1为小量时\\
    &J_r(\phi_1)^{-1}\phi_2+\phi_1\quad当\phi_2为小量时
    \end{aligned}
    \right. 
$$
    
其中左乘雅各比$J_l$为 (<font color=red>怎么推导出来的？</font>)

$$J_l=\frac{sin\theta}{\theta}I+(1-\frac{sin\theta}{\theta})aa^T+\frac{1-cos\theta}{\theta}a^\wedge$$

$$J_l^{-1}=\frac{\theta}{2}cot\frac{\theta}{2}I+(1-\frac{\theta}{2}cot\frac{\theta}{2})aa^T-\frac{\theta}{2}a^\wedge$$

且

$$J_r(\phi)=J_l(-\phi)$$

对于某个旋转$R$的李代数$\phi$，左乘旋转微元$\Delta R$，微元对应的李代数为$\Delta \phi$。李群上旋转量经过微元的作用为$\Delta R\cdot R$，其李代数上根据BCH近似，对应的

$$J_l^{-1}(\phi)\Delta\phi+\phi$$

即，李群上的乘法变为李代数上的加法

$$\exp(\Delta\phi^\wedge)\exp(\phi^\wedge) = \exp((J_l^{-1}(\phi)\Delta\phi+\phi)^\wedge)$$

反之，李代数上的加法会近似为李群上的乘法（将上式用$\Delta\phi$替换$J_l^{-1}(\phi)\Delta\phi$）:

$$\exp((\phi+\Delta\phi)^\wedge) = \exp((J_l(\phi)\Delta\phi)^\wedge)\exp(\phi^\wedge)=\exp(\phi^\wedge)\exp((J_r(\phi)\Delta\phi)^\wedge)$$

同理，$SE(3)$上的类似BCH近似公式

$$\exp(\Delta\xi^\wedge)\exp(\xi^\wedge) = \exp((\mathcal{J}_l^{-1}(\xi)\Delta\xi+\xi)^\wedge)$$

$$\exp(\xi^\wedge)\exp(\Delta\xi^\wedge) = \exp((\mathcal{J}_r^{-1}(\xi)\Delta\xi+\xi)^\wedge)$$

$\mathcal{J}_l$复杂，不写实际形式？

## SO3上李代数的求导

### 李代数求导
对于$SO(3)$的情况，对于一个空间点旋转，得到$Rp$。计算旋转后的点对旋转的导数：

$$\frac{\partial(Rp)}{\partial R}\qquad (不严谨的)$$

设$R$对应的李代数为$\phi$，转为计算

$$
\begin{aligned}
\frac{\partial(\exp(\phi^\wedge)p)}{\partial \phi}
&=\lim\limits_{\delta\phi\to0} \frac{\exp((\phi+\delta\phi)^\wedge)p-\exp(\phi^\wedge)p}{\delta\phi}\\
&=\lim\limits_{\delta\phi\to0} \frac{\exp((J_l(\phi)\delta\phi)^\wedge)\exp(\phi^\wedge)p-\exp(\phi^\wedge)p}{\delta\phi}\\
\end{aligned}
$$

又由于$\exp((J_l(\phi)\delta\phi)^\wedge)=I+(J_l(\phi)\delta\phi)^\wedge+\frac{1}{2}((J_l(\phi)\delta\phi)^\wedge)^2+\cdots$取线性部分近似，有

$$
\begin{aligned}
&\approx\lim\limits_{\delta\phi\to0} \frac{(I+(J_l(\phi)\delta\phi)^\wedge)\exp(\phi^\wedge)p-\exp(\phi^\wedge)p}{\delta\phi}\\
&=\lim\limits_{\delta\phi\to0}\frac{(J_l(\phi)\delta\phi)^\wedge\exp(\phi^\wedge)p}{\delta\phi}\\
&=\lim\limits_{\delta\phi\to0}\frac{-(\exp(\phi^\wedge)p)^\wedge(J_l(\phi)\delta\phi)}{\delta\phi} (property\ of\ cross-product\ exchange)\\
&=-(Rp)^\wedge J_l.
\end{aligned}
$$

### 扰动模型（左）

相当于对矩阵$R$左乘一个微扰$\delta R$，两者李代数分别为$\phi,\delta\phi$，有

$$
\begin{aligned}
\frac{\partial(\exp(\phi^\wedge)p)}{\partial \phi}
&=\lim\limits_{\delta\phi\to0} \frac{\exp(\delta\phi^\wedge)\exp(\phi^\wedge)p-\exp(\phi^\wedge)p}{\delta\phi}\\
&\approx\lim\limits_{\delta\phi\to0} \frac{(I+\delta\phi^\wedge)\exp(\phi^\wedge)p-\exp(\phi^\wedge)p}{\delta\phi}\\
&=\lim\limits_{\delta\phi\to0}\frac{\delta\phi^\wedge\exp(\phi^\wedge)p}{\delta\phi}\\
&=\lim\limits_{\delta\phi\to0}\frac{-(\exp(\phi^\wedge)p)^\wedge\delta\phi}{\delta\phi}\\
&=-(Rp)^\wedge.
\end{aligned}
$$

计算更简单，不用算雅各比了！

### $SE(3)$上的李代数求导，左扰动

空间点$p$,经过变换$T$,李代数$\xi$,扰动$\Delta T$,李代数$\delta\xi$.

$$
\begin{aligned}
\frac{\partial(Tp)}{\partial\delta\xi}
&\approx\lim\limits_{\delta\xi\to0} \frac{\delta\xi^\wedge\exp(\xi^\wedge)p}{\delta\xi}\\
&=\lim\limits_{\delta\xi\to0} \frac{\begin{bmatrix}\delta\phi^\wedge&\delta\rho\\0^T&0\end{bmatrix}\begin{bmatrix}Rp+t\\1\end{bmatrix}}{\delta\xi}\\
&=\lim\limits_{\delta\xi\to0} \frac{\begin{bmatrix}\delta\phi^\wedge(Rp+t)+\delta\rho\\0\end{bmatrix}}{\delta\xi}\\
\end{aligned}
$$

矩阵这里的除法意义即存在$4\times6$的矩阵$M$满足

$$\begin{bmatrix}\delta\phi^\wedge(Rp+t)+\delta\rho\\0\end{bmatrix}=M\begin{bmatrix}\delta\rho \\ \delta\phi\end{bmatrix}$$

因此利用分块矩阵设未知数，可以求出$M$

$$\frac{\partial(Tp)}{\partial\delta\xi} = \begin{bmatrix}I & -(Rp+t)^\wedge \\ 0^T & 0^T\end{bmatrix}\triangleq(Tp)^\odot$$
